# Data preparation and preprocessing

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import datetime as dt
from pytz import timezone
from tqdm import tqdm


In [2]:
repertory = './data/'

In [ ]:
file = 'ar41_for_ulb.csv'
N = 2**13
B = 2**6
df = pd.read_csv(repertory+file, sep=';')

## Add weather data to csv file

#### Get weather data from openweathermap

In [ ]:
# from urllib.request import urlopen
# import json
# import csv
# import time
# import datetime


# # Timestamp 1 january 2023 UTC  is	1672531201
# # Each hour, + 3600
# # Each day, + 86400
# # End, 15 september, UTC is 1694818801
# # Duration : 258 days or 22287600 seconds
# # Careful, changement d'heure happening in Belgium but not for UTC

# file = 'csvWeather.csv'
# with open(repertory+file,'w',newline='') as weather :
#     writer = csv.writer(weather)
#     # Write the columns names on the csv
#     fields = ['date','time','temp','wind','humidity']
#     writer.writerow(fields)
#     # For Bxl, Gand and Bastogne, write temperature, wind speed and humidity for each hour and each day between 01/01/23 and 15/09/23
#     for day in range(258):
#         for hour in range(24):
#             urlbx = 'https://history.openweathermap.org/data/2.5/history/city?lat=50.5&lon=4.20&start={}&cnt=1&appid=0ecabdca455f4f022ca58f1a5929a9b7'.format(1672531201+(day*86400)+(hour*3600))
#             data = json.load(urlopen(urlbx))
#             windspeed = data['list'][0]['wind']['speed']
#             temp = data['list'][0]['main']['temp']
#             humidity =  data['list'][0]['main']['humidity']
#             dt = datetime.datetime.fromtimestamp(data['list'][0]['dt'])
#             dt = str(dt).split()
#             writer.writerow([dt[0],dt[1],temp,windspeed,humidity])

#             urlgand = 'https://history.openweathermap.org/data/2.5/history/city?lat=51.03&lon=3.43&start={}&cnt=1&appid=0ecabdca455f4f022ca58f1a5929a9b7'.format(1672531201+(day*86400)+(hour*3600))
#             data = json.load(urlopen(urlgand))
#             windspeed = data['list'][0]['wind']['speed']
#             temp = data['list'][0]['main']['temp']
#             humidity =  data['list'][0]['main']['humidity']
#             writer.writerow([dt[0],dt[1],temp,windspeed,humidity])

#             urlbast = 'https://history.openweathermap.org/data/2.5/history/city?lat=50.0&lon=5.43&start={}&cnt=1&appid=0ecabdca455f4f022ca58f1a5929a9b7'.format(1672531201+(day*86400)+(hour*3600))
#             data = json.load(urlopen(urlbast))
#             windspeed = data['list'][0]['wind']['speed']
#             temp = data['list'][0]['main']['temp']
#             humidity =  data['list'][0]['main']['humidity']
#             writer.writerow([dt[0],dt[1],temp,windspeed,humidity])

#### Combine our dataset with weather data 

In [ ]:
# Find the closest point to define which weather report is the most relevant 
def closest(point, locs):
    close = 9e20
    for j in locs :
        dist = 0
        for i in range (2) :
            dist += (point[i]-j[i])**2
        if np.sqrt(dist) < close :
            close = np.sqrt(dist)
            best = j
    return best

In [ ]:
pathWeather = repertory+'csvWeather.csv'
pathTrain = repertory+'ar41_for_ulb.csv'
weather = pd.read_csv(pathWeather)
train = pd.read_csv(pathTrain,sep=';')

In [ ]:
# Creating a series for each dataframe that contains the closest point of the 3 studied locations
# We chose 3 different towns in Belgium based on their average temperatures. 
# One in the south, rather cold, one in the center and one in the north, rather warm.
locs = [(50.5, 4.2),(51.0,3.43),(50.0,5.43)]
weather['closest'] = list(zip(weather['lat'], weather['lon']))
train['coord'] = list(zip(train['lat'], train['lon']))
train['closest'] = train.apply(lambda row: closest(row['coord'],locs),axis=1)

In [ ]:
train['date'] = train['timestamps_UTC'].str.split().str[0]
train['time'] = train['timestamps_UTC'].str.split().str[1].str[:2]
weather['time'] = pd.to_datetime(weather['time'], format='%H:%M:%S')
weather['time'] = weather['time'].dt.hour.apply(lambda x: str(x).zfill(2))
print(train.shape,weather.shape)
final = pd.merge(train,weather,on=['time','date','closest'],how='inner')
print(final)

In [ ]:
writer = pd.ExcelWriter('final2.xlsx',engine='xlsxwriter') #TODO
final2 = final[final['mapped_veh_id'] < 105]
final2.to_excel(writer,sheet_name='sheet1')
writer.close()
final.to_csv(repertory+'trains_weather.csv',sep=";")

## Filter impossible values

In [ ]:
#Read csv with weather data
file = 'trains_weather.csv'
N = 17679273
df = pd.read_csv(repertory+file, sep=';')
df.info()
df = df.iloc[:, :-2].drop(df.columns[16:18], axis=1) #remove some useless columns (closest,lat_y,lon_y)

In [ ]:
'''use file to make fit & ruled_out through rule-based processing.
fitting is not sorted by trains or by time'''

def sep_temp_air(df) :
    upper = 200
    lower = -15 # Determine if it is an overflow (and should be removed) or not
    high = (df["RS_E_InAirTemp_PC1"] > upper) | (df['RS_E_InAirTemp_PC2'] > upper)
    higher = df.copy()[high] #Getting only rows that show an upper error
    higher['hta'] = True #Adding a column to indicate if there's an upper error

    low = (df["RS_E_InAirTemp_PC1"] < lower) | (df['RS_E_InAirTemp_PC2'] < lower)
    lower = df.copy()[low]
    lower['lta'] = True

    fitting = df[~(high | low)]
    return fitting, higher, lower

def sep_temp_water(df) :
    upper = 100
    lower = -5 # Determine if it is an overflow (and should be removed) or not
    high = (df["RS_E_WatTemp_PC1"] > upper) | (df['RS_E_WatTemp_PC2'] > upper)
    higher = df.copy()[high]
    higher['htw'] = True
    low = (df["RS_E_WatTemp_PC1"] <= lower) | (df['RS_E_WatTemp_PC2'] <= lower)
    lower = df.copy()[low]
    lower['ltw'] = True
    fitting = df[~(high | low)]
    return fitting, higher, lower

def sep_temp_oil(df) :
    upper = 200
    lower = -100 # Determine if it is an overflow (and should be removed) or not
    high = (df["RS_T_OilTemp_PC1"] > upper) | (df['RS_T_OilTemp_PC2'] > upper)
    higher = df.copy()[high]
    higher['hto'] = True
    low = (df["RS_T_OilTemp_PC1"] < lower) | (df['RS_T_OilTemp_PC2'] < lower)
    lower = df.copy()[low]
    lower['lto'] = True
    fitting = df[~(high | low)]
    return fitting, higher, lower

def sep_rpm(df) :
    upper = 3000
    lower = -1
    high = (df["RS_E_RPM_PC1"] > upper) | (df['RS_E_RPM_PC2'] > upper)
    higher = df.copy()[high]
    higher['hrpm'] = True
    low = (df["RS_E_RPM_PC1"] < lower) | (df['RS_E_RPM_PC2'] < lower)
    lower = df.copy()[low]
    lower['lrpm'] = True
    fitting = df[~(high | low)]
    return fitting, higher, lower

def sep_press(df) :
    upper = 689
    lower = 1 
    high = (df["RS_E_OilPress_PC1"] > upper) | (df['RS_E_OilPress_PC2'] > upper)
    higher = df.copy()[high]
    higher['hpress'] = True
    low = (df["RS_E_OilPress_PC1"] < lower) | (df['RS_E_OilPress_PC2'] < lower)
    lower = df.copy()[low]
    lower['lpress'] = True
    fitting = df[~(high | low)]
    return fitting, higher, lower

file = 'trains_weather.csv'

df = pd.read_csv(repertory+ file,sep=";",index_col=[0])   
print(df.head())

N = df.shape[0]
df = df.dropna() #12000 NaN's in the PC2 columns
print("Naaan : ", df.shape[0],N-df.shape[0])
ruled_out = df.isna()


fit,h,l = sep_temp_air(df)
print("Temperature air : ", fit.shape, h.shape, l.shape)
ruled_out = ruled_out.append(h)
ruled_out = ruled_out.append(l)

fit,h,l = sep_temp_oil(fit)
print("Temperature oil : ", fit.shape, h.shape, l.shape)
ruled_out = ruled_out.append(h)
ruled_out = ruled_out.append(l)

fit,h,l = sep_temp_water(fit)
print("Temperature water : ", fit.shape, h.shape, l.shape)
ruled_out = ruled_out.append(h)
ruled_out = ruled_out.append(l)

fit,h,l = sep_press(fit)
print("Pressure : ", fit.shape, h.shape, l.shape)
ruled_out = ruled_out.append(h)
ruled_out = ruled_out.append(l)

fit,h,l = sep_rpm(fit)
print("RPM : ", fit.shape, h.shape, l.shape)
ruled_out = ruled_out.append(h)
ruled_out = ruled_out.append(l)


In [ ]:
fit.to_csv(repertory+'fittingtst.csv',sep=';',index=False)
ruled_out.to_csv(repertory+'ruled_outtst.csv',sep=';',index=False)

In [ ]:
df = fit

In [3]:
file = 'fittingtst.csv'

df = pd.read_csv(repertory+file, sep=';')

## Visualize time difference 

In [ ]:
#convert timezone
df['timestamps_UTC'] = pd.to_datetime(df['timestamps_UTC'], utc=True)
tzone = timezone('Europe/Paris')
df['timestamps_UTC'] = df['timestamps_UTC'].dt.tz_convert(tzone)

#remove nan
print(df.shape)
df = df.dropna()
print(df.shape)

df = df.sort_values(by=['mapped_veh_id','timestamps_UTC'])
time_diffs = df['timestamps_UTC'].diff().dt.total_seconds()

custom_bins = np.concatenate([np.arange(0, 130, 1)])

# Plot a histogram of the time differences with custom bin edges
plt.hist(time_diffs, bins=custom_bins, edgecolor='black')
plt.yscale("log")
plt.xlabel('Time Differences (seconds)')
plt.ylabel('Frequency')
plt.title('Histogram of Time Differences')
plt.show()

## Create time series by train and with regular interval of time between two consecutive timestamps

In [4]:
# Sort the DataFrame by train_id and timestamps in chronological order
df = df.sort_values(by=['mapped_veh_id','timestamps_UTC']).reset_index()

# Convert timestamps to datetime objects
df['timestamps_UTC'] = pd.to_datetime(df['timestamps_UTC'], utc=True)

# Calculate time differences (T(n)) between consecutive samples and fill NaN with 0
n_n1 = df['timestamps_UTC'].diff().dt.total_seconds()
n_n1 = n_n1.to_frame(name='n_n1')
n_n1 = n_n1.fillna(0)

# Insert the calculated time differences as a new column 'n_n1' at position 3
df.insert(3,'n_n1',n_n1)

df2 = df.copy()


In [6]:
import pytz
from datetime import timezone

now = pd.datetime.now(tz=pytz.UTC)

df2['bin_interval_time'] = ((df2['timestamps_UTC'] - now).dt.total_seconds() % 60).astype(int) #TODO check
print(df2.head(100))


# Sort the DataFrame by train_id and timestamps again
sortedByTrain2 = df2.sort_values(by=['mapped_veh_id','timestamps_UTC']).reset_index()

# Convert timestamps to datetime objects
sortedByTrain2['timestamps_UTC'] = pd.to_datetime(sortedByTrain2['timestamps_UTC'], utc=True)

# Add a 'sequence_start' column based on time gaps greater than 150 seconds
sortedByTrain2['sequence_start'] = sortedByTrain2['timestamps_UTC'].diff().dt.total_seconds() > 150  #Can we not use the n__n1 column?

sortedByTrain2['traj'] = sortedByTrain2['sequence_start'].cumsum()

sortedByTrain2 = sortedByTrain2.drop(['sequence_start','date','time'],axis = 1)

print(sortedByTrain2[['traj','timestamps_UTC','n_n1']].head(100))

sortedByTrain2['most_common_bin'] = sortedByTrain2.groupby('traj')['bin_interval_time'].transform(lambda x: x.mode().iloc[0])

print(df2.head(100))

C:\Users\Vince\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:4: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  after removing the cwd from sys.path.


       index  Unnamed: 0.1  mapped_veh_id  n_n1            timestamps_UTC  \
0   10519630       5493376          102.0   0.0 2023-01-23 07:25:08+00:00   
1   10519642       8969009          102.0   8.0 2023-01-23 07:25:16+00:00   
2   10519662      13873566          102.0  21.0 2023-01-23 07:25:37+00:00   
3   10519663      14994675          102.0   4.0 2023-01-23 07:25:41+00:00   
4   10519656      11935795          102.0  29.0 2023-01-23 07:26:10+00:00   
..       ...           ...            ...   ...                       ...   
95  10539750       3691099          102.0   6.0 2023-01-23 14:52:14+00:00   
96  10540524       8960631          102.0  54.0 2023-01-23 14:53:08+00:00   
97  10539849       4395399          102.0   6.0 2023-01-23 14:53:14+00:00   
98  10539500       2406388          102.0  55.0 2023-01-23 14:54:09+00:00   
99  10541729      16102555          102.0   6.0 2023-01-23 14:54:15+00:00   

        lat_x     lon_x  RS_E_InAirTemp_PC1  RS_E_InAirTemp_PC2  \
0   51.0

In [7]:
sortedByTrain2.drop("n_n1", inplace=True, axis = 1)
print(sortedByTrain2.columns)
# Sort the DataFrame by train_id and timestamps in chronological order
sortedByTrain2 = sortedByTrain2.sort_values(by=['mapped_veh_id','timestamps_UTC'])

# Convert timestamps to datetime objects
sortedByTrain2['timestamps_UTC'] = pd.to_datetime(sortedByTrain2['timestamps_UTC'], utc=True)

# Calculate time differences (T(n)) between consecutive samples and fill NaN with 0
n_n1 = sortedByTrain2['timestamps_UTC'].diff().dt.total_seconds()
n_n1 = n_n1.to_frame(name='n_n1')
n_n1 = n_n1.fillna(0)

# Insert the calculated time differences as a new column 'n_n1' at position 3
sortedByTrain2.insert(3,'n_n1',n_n1)

Index(['level_0', 'index', 'Unnamed: 0.1', 'mapped_veh_id', 'timestamps_UTC',
       'lat_x', 'lon_x', 'RS_E_InAirTemp_PC1', 'RS_E_InAirTemp_PC2',
       'RS_E_OilPress_PC1', 'RS_E_OilPress_PC2', 'RS_E_RPM_PC1',
       'RS_E_RPM_PC2', 'RS_E_WatTemp_PC1', 'RS_E_WatTemp_PC2',
       'RS_T_OilTemp_PC1', 'RS_T_OilTemp_PC2', 'coord', 'closest', 'temp',
       'wind', 'humidity', 'lat_y', 'lon_y', 'bin_interval_time', 'traj',
       'most_common_bin'],
      dtype='object')


In [8]:
print(sortedByTrain2[['traj','timestamps_UTC','n_n1','bin_interval_time','most_common_bin']].head(100))

    traj            timestamps_UTC  n_n1  bin_interval_time  most_common_bin
0      0 2023-01-23 07:25:08+00:00   0.0                 23               23
1      0 2023-01-23 07:25:16+00:00   8.0                 31               23
2      0 2023-01-23 07:25:37+00:00  21.0                 52               23
3      0 2023-01-23 07:25:41+00:00   4.0                 56               23
4      0 2023-01-23 07:26:10+00:00  29.0                 25               23
..   ...                       ...   ...                ...              ...
95     1 2023-01-23 14:52:14+00:00   6.0                 29               27
96     1 2023-01-23 14:53:08+00:00  54.0                 23               27
97     1 2023-01-23 14:53:14+00:00   6.0                 29               27
98     1 2023-01-23 14:54:09+00:00  55.0                 24               27
99     1 2023-01-23 14:54:15+00:00   6.0                 30               27

[100 rows x 5 columns]


In [9]:
tmp = sortedByTrain2[sortedByTrain2["traj"]==20200]
print(tmp[["timestamps_UTC", "n_n1", "bin_interval_time","most_common_bin"]].head(30))

                   timestamps_UTC    n_n1  bin_interval_time  most_common_bin
7235603 2023-06-02 14:03:30+00:00  2109.0                 45               45
7235604 2023-06-02 14:04:30+00:00    60.0                 45               45
7235605 2023-06-02 14:05:10+00:00    40.0                 25               45
7235606 2023-06-02 14:05:30+00:00    20.0                 45               45
7235607 2023-06-02 14:06:30+00:00    60.0                 45               45
7235608 2023-06-02 14:06:40+00:00    10.0                 55               45
7235609 2023-06-02 14:06:41+00:00     1.0                 56               45
7235610 2023-06-02 14:06:50+00:00     9.0                  5               45
7235611 2023-06-02 14:06:51+00:00     1.0                  6               45
7235612 2023-06-02 14:07:30+00:00    39.0                 45               45
7235613 2023-06-02 14:07:31+00:00     1.0                 46               45
7235614 2023-06-02 14:08:31+00:00    60.0                 46    

In [10]:
df3 = sortedByTrain2.copy()
tau = 10
sortedByTrain2 = sortedByTrain2[((sortedByTrain2['bin_interval_time'] >= sortedByTrain2['most_common_bin']-tau) & (sortedByTrain2['bin_interval_time'] <= sortedByTrain2['most_common_bin']+tau))]

In [11]:
sortedByTrain2.drop("n_n1", inplace=True, axis = 1)
print(sortedByTrain2.columns)
# Sort the DataFrame by train_id and timestamps in chronological order
sortedByTrain2 = sortedByTrain2.sort_values(by=['mapped_veh_id','timestamps_UTC'])

# Convert timestamps to datetime objects
sortedByTrain2['timestamps_UTC'] = pd.to_datetime(sortedByTrain2['timestamps_UTC'], utc=True)

# Calculate time differences (T(n)) between consecutive samples and fill NaN with 0
n_n1 = sortedByTrain2['timestamps_UTC'].diff().dt.total_seconds()
n_n1 = n_n1.to_frame(name='n_n1')
n_n1 = n_n1.fillna(0)

# Insert the calculated time differences as a new column 'n_n1' at position 3
sortedByTrain2.insert(3,'n_n1',n_n1)

Index(['level_0', 'index', 'Unnamed: 0.1', 'mapped_veh_id', 'timestamps_UTC',
       'lat_x', 'lon_x', 'RS_E_InAirTemp_PC1', 'RS_E_InAirTemp_PC2',
       'RS_E_OilPress_PC1', 'RS_E_OilPress_PC2', 'RS_E_RPM_PC1',
       'RS_E_RPM_PC2', 'RS_E_WatTemp_PC1', 'RS_E_WatTemp_PC2',
       'RS_T_OilTemp_PC1', 'RS_T_OilTemp_PC2', 'coord', 'closest', 'temp',
       'wind', 'humidity', 'lat_y', 'lon_y', 'bin_interval_time', 'traj',
       'most_common_bin'],
      dtype='object')


In [12]:
tmp = sortedByTrain2[sortedByTrain2["traj"]==20200]
print(tmp[["timestamps_UTC", "n_n1", "bin_interval_time","most_common_bin"]].head(30))

                   timestamps_UTC    n_n1  bin_interval_time  most_common_bin
7235603 2023-06-02 14:03:30+00:00  2109.0                 45               45
7235604 2023-06-02 14:04:30+00:00    60.0                 45               45
7235606 2023-06-02 14:05:30+00:00    60.0                 45               45
7235607 2023-06-02 14:06:30+00:00    60.0                 45               45
7235608 2023-06-02 14:06:40+00:00    10.0                 55               45
7235612 2023-06-02 14:07:30+00:00    50.0                 45               45
7235613 2023-06-02 14:07:31+00:00     1.0                 46               45
7235614 2023-06-02 14:08:31+00:00    60.0                 46               45
7235615 2023-06-02 14:08:34+00:00     3.0                 49               45
7235616 2023-06-02 14:09:21+00:00    47.0                 36               45
7235617 2023-06-02 14:09:34+00:00    13.0                 49               45
7235618 2023-06-02 14:10:24+00:00    50.0                 39    

In [14]:
sortedByTrain2.to_csv('./result/clean1.csv',sep=',')

In [ ]:
#drop time intervals 
tau = 10
df = df[((df['n_n1'] >= 60-tau) & (df['n_n1'] <= 60+tau)) | (df['n_n1'] >= 120-tau) & (df['n_n1'] <= 120+tau)]
print(df.shape[0])

In [9]:
print(df.shape[0])
print(sortedByTrain2.shape[0])

16517984
13522586


In [10]:
df_final = pd.DataFrame(columns=sortedByTrain2.columns)
print(df_final)
global add 
add = -1 

def gb_to_rows(gb) :
    gb.apply(lambda w: addrow(w,df_final),axis=1)
    print(gb['traj'].drop_duplicates())
    
def addrow(row,df_final) :
    global add
    if row['n_n1'] > 90 :
        df_final.loc[add,'timestamps_UTC'] = row['timestamps_UTC'] + dt.timedelta(seconds=60)
        df_final.loc[add,'mapped_veh_id'] = row['mapped_veh_id']
        df_final.loc[add,'traj'] = row['traj']
        add -= 1

Empty DataFrame
Columns: [level_0, index, Unnamed: 0.1, n_n1, mapped_veh_id, timestamps_UTC, lat_x, lon_x, RS_E_InAirTemp_PC1, RS_E_InAirTemp_PC2, RS_E_OilPress_PC1, RS_E_OilPress_PC2, RS_E_RPM_PC1, RS_E_RPM_PC2, RS_E_WatTemp_PC1, RS_E_WatTemp_PC2, RS_T_OilTemp_PC1, RS_T_OilTemp_PC2, coord, closest, temp, wind, humidity, lat_y, lon_y, bin_interval_time, traj, most_common_bin]
Index: []

[0 rows x 28 columns]


In [11]:
import datetime as dt
sortedByTrain2.groupby('traj').apply(lambda z: gb_to_rows(z))

0    0
Name: traj, dtype: int32
8    1
Name: traj, dtype: int32
980    2
Name: traj, dtype: int32
992    3
Name: traj, dtype: int32
1892    4
Name: traj, dtype: int32
2024    5
Name: traj, dtype: int32
2032    6
Name: traj, dtype: int32
2052    7
Name: traj, dtype: int32
3671    8
Name: traj, dtype: int32
3672    9
Name: traj, dtype: int32
3681    10
Name: traj, dtype: int32
4107    11
Name: traj, dtype: int32
5614    12
Name: traj, dtype: int32
6510    13
Name: traj, dtype: int32
6531    14
Name: traj, dtype: int32
6582    15
Name: traj, dtype: int32
6590    16
Name: traj, dtype: int32
6609    17
Name: traj, dtype: int32
6639    18
Name: traj, dtype: int32
7065    19
Name: traj, dtype: int32
7942    20
Name: traj, dtype: int32
9283    21
Name: traj, dtype: int32
9816    22
Name: traj, dtype: int32
10175    23
Name: traj, dtype: int32
11045    24
Name: traj, dtype: int32
11050    25
Name: traj, dtype: int32
11561    26
Name: traj, dtype: int32
11756    27
Name: traj, dtype: int32
12097

""


In [18]:
tmp = sortedByTrain2[sortedByTrain2["traj"]==20200]
print(tmp[["timestamps_UTC", "n_n1", "bin_interval_time","most_common_bin"]].head(30))

                   timestamps_UTC    n_n1  bin_interval_time  most_common_bin
7235603 2023-06-02 14:03:30+00:00  2109.0                 40               40
7235604 2023-06-02 14:04:30+00:00    60.0                 40               40
7235606 2023-06-02 14:05:30+00:00    20.0                 40               40
7235607 2023-06-02 14:06:30+00:00    60.0                 40               40
7235608 2023-06-02 14:06:40+00:00    10.0                 50               40
7235612 2023-06-02 14:07:30+00:00    39.0                 40               40
7235613 2023-06-02 14:07:31+00:00     1.0                 41               40
7235614 2023-06-02 14:08:31+00:00    60.0                 41               40
7235615 2023-06-02 14:08:34+00:00     3.0                 44               40
7235616 2023-06-02 14:09:21+00:00    47.0                 31               40
7235617 2023-06-02 14:09:34+00:00    13.0                 44               40
7235618 2023-06-02 14:10:24+00:00    50.0                 34    

In [12]:
print(sortedByTrain2[sortedByTrain2["traj"]==20000])

         level_0    index  Unnamed: 0.1  n_n1  mapped_veh_id  \
7151904  7151904  5430931      11554806  63.0          145.0   
7151905  7151905  5430916      11537723  60.0          145.0   
7151906  7151906  5430642       4556427  60.0          145.0   
7151907  7151907  5431044      15077462  60.0          145.0   
7151908  7151908  5431121      16490495  61.0          145.0   

                   timestamps_UTC      lat_x     lon_x  RS_E_InAirTemp_PC1  \
7151904 2023-03-14 16:18:50+00:00  50.845651  3.817807                43.0   
7151905 2023-03-14 16:19:50+00:00  50.855806  3.810456                41.0   
7151906 2023-03-14 16:20:50+00:00  50.861938  3.813221                37.0   
7151907 2023-03-14 16:21:50+00:00  50.867112  3.814033                34.0   
7151908 2023-03-14 16:22:51+00:00  50.868716  3.814317                30.0   

         RS_E_InAirTemp_PC2  ...                    coord       closest  \
7151904                36.0  ...  (50.8456507, 3.8178068)  (51.0, 3.43)

In [ ]:
sortedByTrain2 = df3
tau = 12
print(sortedByTrain2.head(100))
sortedByTrain2 = sortedByTrain2[((sortedByTrain2['n_n1'] >= 60-tau) & (sortedByTrain2['n_n1'] <= 60+tau)) | (sortedByTrain2['n_n1'] >= 120-tau) & (sortedByTrain2['n_n1'] <= 120+tau)]
print(sortedByTrain2.head(100))
sortedByTrain2.drop("n_n1", axis = 1,  inplace=True)

In [ ]:
print(sortedByTrain2[['traj','timestamps_UTC','n_n1','bin_interval_time','most_common_bin']].head(100))

In [ ]:

#drop time intervals 
tau = 12
print(df.head(100))
df = df[((df['n_n1'] >= 60-tau) & (df['n_n1'] <= 60+tau)) | (df['n_n1'] >= 120-tau) & (df['n_n1'] <= 120+tau)]
print(df.head(100))
df.drop("n_n1", axis = 1,  inplace=True)

In [ ]:
# Sort the DataFrame by train_id and timestamps again
sortedByTrain = df.sort_values(by=['mapped_veh_id','timestamps_UTC']).reset_index()

# Convert timestamps to datetime objects
sortedByTrain['timestamps_UTC'] = pd.to_datetime(sortedByTrain['timestamps_UTC'], utc=True)

# Calculate time differences (T(n)) between consecutive samples and fill NaN with 0
n_n1 = sortedByTrain['timestamps_UTC'].diff().dt.total_seconds()
n_n1 = n_n1.to_frame(name='n_n1')
n_n1 = n_n1.fillna(0)

# Insert the calculated time differences as a new column 'n_n1' at position 3
sortedByTrain.insert(3,'n_n1',n_n1)

In [ ]:
# Add a 'sequence_start' column based on time gaps greater than 150 seconds
sortedByTrain['sequence_start'] = sortedByTrain['timestamps_UTC'].diff().dt.total_seconds() > 150  #Can we not use the n__n1 column?

sortedByTrain['traj'] = sortedByTrain['sequence_start'].cumsum()
print(sortedByTrain['traj'].drop_duplicates())
sortedByTrain = sortedByTrain.drop(['sequence_start','date','time'],axis = 1)

print(sortedByTrain[['traj','timestamps_UTC','n_n1']].head(100))

In [ ]:
print(sortedByTrain[['traj','timestamps_UTC','n_n1']].head(502))

In [ ]:
df3 = pd.DataFrame(columns=sortedByTrain.columns)
print(df3)
global add 
add = -1 

def gb_to_rows(gb) :
    gb.apply(lambda w: addrow(w,df3),axis=1)
    print(gb['traj'].drop_duplicates())
def addrow(row,df3) :
    global add
    if row['n_n1'] > 90 :
        df3.loc[add,'timestamps_UTC'] = row['timestamps_UTC'] + dt.timedelta(seconds=60)
        df3.loc[add,'mapped_veh_id'] = row['mapped_veh_id']
        df3.loc[add,'traj'] = row['traj']
        add -= 1

In [ ]:
import datetime as dt
sortedByTrain.groupby('traj').apply(lambda z: gb_to_rows(z))

In [ ]:
print(df3.shape)
df = pd.concat([sortedByTrain,df3])
print(sortedByTrain.shape)
print(df.shape)
df=df.interpolate()
print(df.shape)

In [ ]:
sortedByTrain.to_csv(dirpath/'trajets_train.csv',sep=',')

In [ ]:
sortedByTrain = sortedByTrain[['mapped_veh_id','timestamps_UTC','traj']]
meantrajlength = sortedByTrain.groupby('traj').count().agg('mean')
print(meantrajlength)
print(sortedByTrain[sortedByTrain['traj'] == 183182])

## Creation of time series

In [ ]:
# sorting + drop na
# Conversion du fuseau horaire
df['timestamps_UTC'] = pd.to_datetime(df['timestamps_UTC'], utc=True)
tzone = timezone('Europe/Paris')
df['timestamps_UTC'] = df['timestamps_UTC'].dt.tz_convert(tzone)

# Suppression des valeurs manquantes
df = df.dropna()

# Tri du DataFrame par 'mapped_veh_id' puis 'timestamps_UTC'
df = df.sort_values(by=['mapped_veh_id', 'timestamps_UTC'])

### Creation of time series (we consider a new time series when, between two timestamps, the train change or the difference of time is greater than 10 minutes)

In [ ]:

# Initialize variables
time_series_list = []
current_time_series = []
current_time_series.append(df.head(1))

# Use tqdm to add a progress bar
for i in tqdm(range(1, len(df)), desc="Processing Rows", unit=" row"):
    # Check if it's the same train
    if df['mapped_veh_id'].iloc[i] == df['mapped_veh_id'].iloc[i - 1]:
        # Calculate time difference in seconds
        time_diff = (df['timestamps_UTC'].iloc[i] - df['timestamps_UTC'].iloc[i - 1]).total_seconds()

        # Check for gaps greater than 5 minutes
        if time_diff > 300:
            # If gap is too large, start a new time series
            if current_time_series:
                time_series_list.append(current_time_series)
            current_time_series = []
        else:
            current_time_series.append(df.iloc(i))
    # Create new time series if it is a new train
    else : 
        if current_time_series:
            time_series_list.append(current_time_series)
        current_time_series = []


In [ ]:
import pandas as pd
from pytz import timezone
from tqdm import tqdm

# Assuming df is your DataFrame

# sorting + drop na
# Conversion du fuseau horaire
df['timestamps_UTC'] = pd.to_datetime(df['timestamps_UTC'], utc=True)
tzone = timezone('Europe/Paris')
df['timestamps_UTC'] = df['timestamps_UTC'].dt.tz_convert(tzone)

# Suppression des valeurs manquantes
df = df.dropna()

# Tri du DataFrame par 'mapped_veh_id' puis 'timestamps_UTC'
df = df.sort_values(by=['mapped_veh_id', 'timestamps_UTC'])

# Initialize variables
time_series_list = []
current_time_series = []
current_time_series.append(df.head(1))

# Iterate over possible starting points (offsets)
max_timestamps = 0
best_offset = 0

for offset in tqdm(range(60), desc="Searching for Best Offset", unit="second"):
    current_time_series = []
    current_time_series.append(df.head(1))

    # Use tqdm to add a progress bar
    for i in range(1, len(df)):
        # Calculate time difference in seconds with the offset
        time_diff = (df['timestamps_UTC'].iloc[i] - df['timestamps_UTC'].iloc[i - 1]).total_seconds() + offset

        # Check for gaps greater than 60 seconds
        if time_diff > 60:
            # If gap is too large, start a new time series
            if current_time_series:
                time_series_list.append(current_time_series)
            current_time_series = []
        else:
            current_time_series.append(df.iloc[i])

    # Create new time series if it is a new train
    if current_time_series:
        time_series_list.append(current_time_series)

    # Update best offset if the current offset gives more timestamps
    if len(current_time_series) > max_timestamps:
        max_timestamps = len(current_time_series)
        best_offset = offset

print(f"Best Offset: {best_offset}")


In [ ]:
for time_series in time_series_list:
    for i in range(0, 59):
        
        for df_chunk in time_series:

## Clean and sort dataset 

In [ ]:
def sep_temp_air(df) :
    upper = 80
    lower = 0 # Determine if it is an overflow (and should be removed) or not
    high = (df["RS_E_InAirTemp_PC1"] > upper) | (df['RS_E_InAirTemp_PC2'] > upper)
    higher = df.copy()[high] #Getting only rows that show an upper error
    higher['hta'] = True #Adding a column to indicate if there's an upper error

    low = (df["RS_E_InAirTemp_PC1"] < lower) | (df['RS_E_InAirTemp_PC2'] < lower)
    lower = df.copy()[low]
    lower['lta'] = True

    fitting = df[~(high | low)]
    return fitting, higher, lower

def sep_temp_water(df) :
    upper = 100
    lower = 0 # Determine if it is an overflow (and should be removed) or not
    high = (df["RS_E_WatTemp_PC1"] > upper) | (df['RS_E_WatTemp_PC2'] > upper)
    higher = df.copy()[high]
    higher['htw'] = True
    low = (df["RS_E_WatTemp_PC1"] <= lower) | (df['RS_E_WatTemp_PC2'] <= lower)
    lower = df.copy()[low]
    lower['ltw'] = True
    fitting = df[~(high | low)]
    return fitting, higher, lower

def sep_temp_oil(df) :
    upper = 100
    lower = 0 # Determine if it is an overflow (and should be removed) or not
    high = (df["RS_T_OilTemp_PC1"] > upper) | (df['RS_T_OilTemp_PC2'] > upper)
    higher = df.copy()[high]
    higher['hto'] = True
    low = (df["RS_T_OilTemp_PC1"] < lower) | (df['RS_T_OilTemp_PC2'] < lower)
    lower = df.copy()[low]
    lower['lto'] = True
    fitting = df[~(high | low)]
    return fitting, higher, lower

def sep_rpm(df) :
    upper = 3000
    lower = 0
    high = (df["RS_E_RPM_PC1"] > upper) | (df['RS_E_RPM_PC2'] > upper)
    higher = df.copy()[high]
    higher['hrpm'] = True
    low = (df["RS_E_RPM_PC1"] < lower) | (df['RS_E_RPM_PC2'] < lower)
    lower = df.copy()[low]
    lower['lrpm'] = True
    fitting = df[~(high | low)]
    return fitting, higher, lower

def sep_press(df) :
    upper = 600
    lower = 100 
    high = (df["RS_E_OilPress_PC1"] > upper) | (df['RS_E_OilPress_PC2'] > upper)
    higher = df.copy()[high]
    higher['hpress'] = True
    low = (df["RS_E_OilPress_PC1"] < lower) | (df['RS_E_OilPress_PC2'] < lower)
    lower = df.copy()[low]
    lower['lpress'] = True
    fitting = df[~(high | low)]
    return fitting, higher, lower


def sep_cap_diff(df) :
    column_name=df.columns[4:]
    sensor_columns = column_name
    sensor_columnsPC1 = sensor_columns[::2]
    sensor_columnsPC2 = sensor_columns[1::2]
    reldiff = 0.5
    diff = pd.DataFrame()
    allzero = pd.DataFrame()
    fitting = df.copy()
    for i in range(len(sensor_columnsPC1)):    
        close = (np.isclose(fitting[sensor_columnsPC1[i]], \
                           fitting[sensor_columnsPC2[i]], rtol=reldiff) & (fitting[sensor_columnsPC1[i]] != 0) & (fitting[sensor_columnsPC2[i]] != 0))
        invalid = ((fitting[sensor_columnsPC1[i]] == 0) & (fitting[sensor_columnsPC2[i]] == 0))
        diff = diff._append(fitting[~close])
        allzero = allzero._append(fitting[invalid])
        fitting = fitting[close & ~invalid]
        print("dshape : ",  diff.shape)
    diff['different'] = True
    allzero['allzero'] = True
    return fitting, diff, allzero
    
df = pd.read_csv(file,sep=";",index_col=[0])   
print(df.head())


fit,h,l = sep_temp_air(df)
print("Temperature air : ", fit.shape, h.shape, l.shape)
ruled_out = h
ruled_out = ruled_out._append(l)

fit,h,l = sep_temp_oil(fit)
print("Temperature oil : ", fit.shape, h.shape, l.shape)
ruled_out = ruled_out._append(h)
ruled_out = ruled_out._append(l)

fit,h,l = sep_temp_water(fit)
print("Temperature water : ", fit.shape, h.shape, l.shape)
ruled_out = ruled_out._append(h)
ruled_out = ruled_out._append(l)

fit,h,l = sep_press(fit)
print("Pressure : ", fit.shape, h.shape, l.shape)
ruled_out = ruled_out._append(h)
ruled_out = ruled_out._append(l)

fit,h,l = sep_rpm(fit)
print("RPM : ", fit.shape, h.shape, l.shape)
ruled_out = ruled_out._append(h)
ruled_out = ruled_out._append(l)


fit, diff, az = sep_cap_diff(fit)
print("Sensor diff : ",fit.shape, diff.shape, az.shape)
ruled_out= ruled_out._append(diff)
ruled_out= ruled_out._append(az)
ruled_out = ruled_out[~ruled_out.index.duplicated(keep='first')]
print("ruled out df : ",ruled_out.shape)

fit.to_csv(dirpath/'fitting.csv',sep=';')
ruled_out.to_csv(dirpath/'ruled_out.csv',sep=';')


In [ ]:
# Convert the 'timestamps_UTC' column to datetime type
df['timestamps_UTC'] = pd.to_datetime(df['timestamps_UTC'])

# Sort the DataFrame by the 'timestamps_UTC' column
df_sorted = df.sort_values(by='timestamps_UTC')

# new_file = 'sorted_ar41_for_ulb.csv'

# # Save the sorted DataFrame to a new CSV file
# df_sorted.to_csv(repertory+new_file, index=False)

In [ ]:
# df_sorted = df
# Select the first 100 rows
df_subset = df_sorted.head(N)

# Check for missing values in each column
missing_values_column = df_subset.isnull().sum()

# Check if there are any missing values in the entire DataFrame
any_missing_values = df_subset.isnull().values.any()

total_missing_values = missing_values_column.sum()

# Display the results
print("Missing values in each column:\n", missing_values_column)
print("\nAre there any missing values in the DataFrame?", any_missing_values)
print("Nombre total de valeurs manquantes dans le DataFrame:", total_missing_values)


In [ ]:
df_sub_subset = df_subset.iloc[:1000]

current_set = df_sub_subset
# Interpolate missing values using linear interpolation
df_interpolated = current_set.interpolate()

# Check if there are any missing values after interpolation
any_missing_values_after_interpolation = df_interpolated.isnull().values.any()

# Display the results
print("Are there any missing values in the DataFrame after interpolation?", any_missing_values_after_interpolation)


In [ ]:
# Interpolate missing values using polynomial interpolation of order 2
df_interpolated_polynomial = current_set.interpolate(method='polynomial', order=2)


In [ ]:
# Plotting
plt.figure(figsize=(10, 6))

# Original Data
plt.scatter(range(len(current_set)), current_set['RS_E_InAirTemp_PC2'], label='Original', marker='o', color='blue', alpha=0.5)

# Linear Interpolation
# plt.plot(range(len(df_interpolated)), df_interpolated['RS_E_InAirTemp_PC2'], label='Linear Interpolation', color='green')

# Polynomial Interpolation
plt.plot(range(len(df_interpolated_polynomial)), df_interpolated_polynomial['RS_E_InAirTemp_PC2'], label='Polynomial Interpolation', color='orange')

# Customize the plot
plt.xlabel('Row number')
plt.ylabel('RS_E_InAirTemp_PC2')
plt.title('Comparison of Interpolations')
plt.legend()
plt.show()

In [ ]:
# Calculate residuals
residuals_linear = current_set['RS_E_InAirTemp_PC2'] - df_interpolated['RS_E_InAirTemp_PC2']
residuals_polynomial = current_set['RS_E_InAirTemp_PC2'] - df_interpolated_polynomial['RS_E_InAirTemp_PC2']

# Plot residuals
plt.figure(figsize=(10, 6))

plt.plot(range(len(current_set)), residuals_linear, label='Linear Residuals', color='green')
plt.plot(range(len(current_set)), residuals_polynomial, label='Polynomial Residuals', color='orange')

plt.axhline(y=0, color='red', linestyle='--', linewidth=1, label='Zero Residual')

plt.xlabel('Timestamp')
plt.ylabel('Residuals')
plt.title('Residuals Comparison')
plt.legend()
plt.show()

In [ ]:
unique_count = df['mapped_veh_id'].nunique()
print(f'Total number of unique values: {unique_count}')

## Standardization of the data

## Autocorrelation

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
def plot_autocorrelation(df, column_name, lags=40):
    """
    Plot autocorrelation for a specific column in a DataFrame.

    Parameters:
    - df (pd.DataFrame): The DataFrame containing the time series data.
    - column_name (str): The name of the column for which to plot the autocorrelation.
    - lags (int): Number of lags to include in the plot.

    Returns:
    None
    """
    plt.figure(figsize=(10, 6))
    plot_acf(df[column_name], lags=lags)
    plt.title(f'Autocorrelation Plot for {column_name}')
    plt.xlabel('Lag')
    plt.ylabel('Autocorrelation')
    plt.show()

selected_columns = current_set.columns[5:]
print(selected_columns)

# selected_columns = current_set.columns[6]
for column_name in selected_columns:
    plot_autocorrelation(current_set, column_name)

## Check stationarity or non stationarity of time series

In [ ]:
# Columns from 5th to 8th (index 4 to 7)
selected_columns = current_set.columns[5:]

for column_name in selected_columns:
    plt.figure(figsize=(10, 6))
    plt.plot(current_set.index, current_set[column_name], label=column_name)
    plt.title(f'Data Plot for {column_name}')
    plt.xlabel('Index')  # Use 'Index' instead of 'Timestamp'
    plt.ylabel(column_name)
    plt.legend()
    plt.show()